In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import psycopg2
from geopy.distance import distance as geo_distance
import geopandas as gpd
# import shapely
import pdb
from tqdm import tqdm
# import time.strftime
import matplotlib as mpl
# import seaborn as sns
mpl.rcParams['axes.linewidth'] = 3

# Analysis idea:

Loop over each intersection. Extract the num crashes within a certain radius. Calculate the number of crashes per yea for each unique intersection. Use a decision tree that uses a gini index on just the num-legs, angle data or some other simple model As we gather more quality feature data more sophisticated methods can replace the decision tree. Use the standard deviation of poisson distribution to calculate the confidence interval.

In [ ]:
POSTGRES_DB= 'rws'
POSTGRES_PASSWORD= 'ug_password'
POSTGRES_USER= 'ug_username'
CURRENT_DIR= os.getcwd()
    
conn = psycopg2.connect(f"host=localhost dbname={POSTGRES_DB} user={POSTGRES_USER} password={POSTGRES_PASSWORD} port=5433")


In [ ]:
# cur = conn.cursor()

sql_crashes = f"""SELECT *,ST_AsText(dc.point) as t_point from crashes.dc_indexed as dc """
sql_ints = f"""SELECT * from planet_osm_intersections_alpha """
sql_roads = f"""SELECT * from planet_osm_roads"""

# crashes = cur.fetchall()
# df_int = pd.read_sql_query(sql_ints, conn)
# df_crashes = pd.read_sql_query(sql_crashes, conn)
df_int = gpd.read_postgis(sql_ints, conn, geom_col="point")
df_crashes = gpd.read_postgis(sql_crashes, conn,geom_col="point")
df_roads = gpd.read_postgis(sql_roads, conn, geom_col="way")


# cur.execute(f"""SELECT * , ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({x},{y}), 4326),3857), xsect.point) as dist FROM planet_osm_intersections_alpha as xsect 
# WHERE  ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint({x},{y}), 4326),3857), xsect.point) < {search_radius} 
# ORDER BY dist
#  """)

# The above code pulls data from the postGIS database running in docker

In [ ]:
df_crashes

# Format datatypes and define exposure window
The idea is that the crash data is collected from various sources. By looking at the data it seems there are fairly uniform collections from 2009-2022. So the strategy is going to use this time frame as our exposure time (time we are recording data in DC), and assume all intersection crashes are accurately recorded during this time. 

In [ ]:
print([x for x in df_crashes.columns])

new_dtypes = {"majorinjuries_bicyclist": int,
              "majorinjuries_driver": int,
              "majorinjuries_pedestrian": int,
              "majorinjuriespassenger": int,
              "fatal_driver": int,
              "fatal_pedestrian": int,
              "fatalpassenger": int,
              "fatal_bicyclist": int,
              "num_legs": int
             }
df_crashes = df_crashes.astype(new_dtypes)
# dataframe = dataframe.astype(new_dtypes)


df_crashes['reportdate'] =  pd.to_datetime(df_crashes['reportdate'], format='%Y/%m/%d %H:%M:%S+%f')

In [ ]:
# df_crashes['reportdate'].hist(bins=150)
# plt.yscale('log')
print(df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]['reportdate'].min())
print(df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]['reportdate'].max())
print((df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]['reportdate'].max())-\
      (df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]['reportdate'].min()))
print("We will normalize the crashes to an exposure time of 12.8 +- 0.5 years")

df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]['reportdate'].hist(bins=150)
# plt.yscale('log')

In [ ]:
exposure_time = 12.8
exposure_time_up = 13.3
exposure_time_down = 12.3
df_crashes = df_crashes[(df_crashes['reportdate'] > "2009") &(df_crashes['reportdate'] < "2022") ]
# severe_columns = [x for x in df_crashes.columns if "FATAL" in x.upper() or "MAJOR" in x.upper()]
# df_crashes_severe = df_crashes[ pd.DataFrame.any(df_crashes[severe_columns].astype(int) > 0,axis=1) ]

In [ ]:
# df_crashes_fatal['crash_count'] = 0
major_injury_columns = [x for x in df_crashes.columns if "MAJOR" in x.upper()]
fatal_injury_columns = [x for x in df_crashes.columns if "FATAL" in x.upper()]
# df_crashes[ pd.DataFrame.any(df_crashes[severe_columns].astype(int) > 0,axis=1) ]
major_injury_columns
fatal_injury_columns
# print(df_int.shape)
# print(df_crashes_fatal.shape)
# for i,row in enumerate(df_int.geometry):
#     print(i)
#     df_int.loc[i,'crash_count'] = sum(df_crashes_fatal.geometry.distance(row) < 50)
# severe_columns

# Major calculation section of the notebook - associating crashes to intersections
Loop over intersections and calculate the crash rates for all, severe, and fatal crashes per intersection.

In [ ]:
df_int['crash_rate'] = 0
df_int['major_injury_crash_rate'] = 0
df_int['fatal_crash_rate'] = 0

df_int['distance_weighted_crash_count'] = 0
df_int['involvesBike'] = 0
# df_crashes_severe = df_crashes_severe.sample(2500)
# df_int = df_int.sample(2500)


print(df_int.shape)
# print(df_crashes_severe.shape)
print(df_crashes.shape)
crash_buffer = df_crashes.geometry.buffer(50)

for i,row in tqdm(enumerate(df_int.geometry)):
#     print(i)
#     df_crashes.geometry.buffer(50)
#     pdb.set_trace()
    buffer_index = crash_buffer.contains(row)
#     dist_vector = df_crashes.geometry.distance(row).astype(float)
#     dist_vector = dist_vector.fillna(1000000)
    df_int.loc[i,'crash_rate'] = sum(buffer_index) / exposure_time
    df_int.loc[i,'major_injury_crash_rate'] = len(df_crashes[((buffer_index) & (df_crashes[major_injury_columns].astype(bool).any(axis=1)))]) / exposure_time
    df_int.loc[i,'fatal_crash_rate'] = len(df_crashes[((buffer_index) & (df_crashes[fatal_injury_columns].astype(bool).any(axis=1)))]) / exposure_time

#     pdb.set_trace()
#     if i > 5:
#         break
#     if sum(dist_vector < 50):
#         pdb.set_trace()
#         df_int.loc[i,'distance_weighted_crash_count'] = sum( (dist_vector < 50).apply(int)*(10/(dist_vector+0.0001)) )
    
#     df_int.loc[i,'isFatal'] = sum( (dist_vector < 50).apply(int)*(df_crashes_severe['fatal_bicyclist'].astype(int)+df_crashes_severe['fatal_driver'].astype(int)+df_crashes_severe['fatal_pedestrian'].astype(int)+df_crashes_severe['fatalpassenger'].astype(int)) )
    
df_int['crash_rate_exposure_err_up'] = df_int['crash_rate'] * (exposure_time/exposure_time_up)
df_int['major_injury_crash_rate_exposure_err_up'] = df_int['major_injury_crash_rate'] * (exposure_time/exposure_time_up)
df_int['fatal_crash_rate_exposure_err_up'] = df_int['fatal_crash_rate'] * (exposure_time/exposure_time_up)

df_int['crash_rate_stat_err'] = np.sqrt(df_int['crash_rate']*exposure_time)/exposure_time
df_int['major_injury_crash_rate_stat_err'] = np.sqrt(df_int['major_injury_crash_rate']*exposure_time)/exposure_time
df_int['fatal_crash_rate_stat_err'] = np.sqrt(df_int['fatal_crash_rate']*exposure_time)/exposure_time
    

In [ ]:
# An uncertainty of 0 for poisson statistics is undefined, assume 1 accident over the exposure time: sqrt(1)/12.8
df_int['crash_rate_stat_err'] = df_int['crash_rate_stat_err'].apply(lambda x: x if x>0 else 0.08)
df_int['fatal_crash_rate_stat_err'] = df_int['fatal_crash_rate_stat_err'].apply(lambda x: x if x>0 else 0.08)
df_int['major_injury_crash_rate_stat_err'] = df_int['major_injury_crash_rate_stat_err'].apply(lambda x: x if x>0 else 0.08)

In [ ]:
# df_int.head(50)
df_int.to_csv('df_int_before_modeling.csv')

# Second major calculation - extract road type features from OSM table
This would be much easier if we just had that information in the flat intersection table. We also do some formatting to convert "num_legs" and "road_type" into categorical variables.

In [ ]:
df_int['road_type'] = None

from shapely.geometry import Point, box

def most_common(lst):
    if not lst:
        return "service_road"
    return max(set(lst), key=lst.count)

for i,row in zip(df_int.geometry.index, df_int.geometry):
    print(i)

    #s.sindex.nearest(Point(1, 1))
    dist_v = df_roads.geometry.distance(row).astype(float)
#     print( most_common( list(df_roads[dist_v < 50]['highway'].values)))
    df_int.loc[i,'road_type'] = most_common( list( df_roads[dist_v < 50]['highway'].values ))
    
    
#     if i > 500 : break    
    

In [ ]:
rtypes = ['motorway', "service_road", 'trunk', 'motorway_link', 'primary',
       'primary_link', 'secondary', 'secondary_link', 'path', 'cycleway',
       'trunk_link', 'footway', 'construction']

for rtype in rtypes:
    df_int[rtype] = 0
    
for i,row in df_int.iterrows():
    df_int.loc[i,row['road_type']] = 1
    
legs = ['2_leg','3_leg','4_leg','5_leg','many_leg']
for leg_type in legs:
    df_int[leg_type] = 0
    
for i,row in df_int.iterrows():
    
    if int(float(df_int.loc[i,'num_legs'])) == 2:
        df_int.loc[i,'2_leg'] = 1
    elif int(float(df_int.loc[i,'num_legs'])) == 3:
        df_int.loc[i,'3_leg'] = 1
    elif int(float(df_int.loc[i,'num_legs'])) == 4:
        df_int.loc[i,'4_leg'] = 1
    elif int(float(df_int.loc[i,'num_legs'])) == 5:
        df_int.loc[i,'5_leg'] = 1
    else:
        df_int.loc[i,'many_leg'] = 1

In [ ]:
# df_int.loc[good_index][['num_legs_from_borderalgo','angle','oneway']+rtypes]
df_int['crash_count'] = df_int['crash_rate'] *12.8
df_int['major_injury_crash_count'] = df_int['major_injury_crash_rate']*12.8
df_int['fatal_crash_count'] = df_int['fatal_crash_rate']*12.8

In [ ]:
df_int['num_legs'].isna().any()
df_int['crash_count']

In [ ]:
df_int['crash_count'].mean()

# Simple statistical analysis
Fit to a tweedie distribution. Convert angle to cosine(rads). Don't have to worry too much about overfitting, because N >> M (much more data than parameters -> deterministic solution). 

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import scipy
import pdb

def cast_to_float(row):
    if row:
        return float(row)
    
scaler = StandardScaler()

clf = linear_model.TweedieRegressor(power=1.1, alpha=0.015, fit_intercept=True, link='log')

# good_index = df_int['crash_count'].apply(cast_to_float).dropna().index
# df_int['distance_weighted_crash_count'] = df_int['distance_weighted_crash_count'].apply(float).fillna(0)
# df_int['num_legs_from_borderalgo'] = df_int['num_legs_from_borderalgo'].apply(cast_to_float).fillna(4)
# df_int['angle'] = df_int['angle'].apply(cast_to_float).fillna(0)
# df_int['oneway'] = df_int['oneway'].apply(cast_to_float).fillna(1)

good_index = df_int.index
# good_index = df_int['crash_count'].apply(cast_to_float).dropna().index
print(good_index)
# df_int.loc[good_index]['num_legs_from_borderalgo'] = df_int.loc[good_index]['num_legs_from_borderalgo'].fillna(4)
# good_index = df_int.loc[good_index]['num_legs'].apply(cast_to_float).dropna().index
# good_index = df_int.loc[good_index].dropna()
print(good_index.shape)
Y = df_int.loc[good_index,'crash_count']#.apply(float)
# Y = df_int.loc[good_index,'crash_'].apply(float)


X = df_int.loc[good_index][['num_legs','angle','oneway']+rtypes+legs].astype(float)
X['angle'] = np.cos(X['angle'] * (np.pi/180))
print(Y.shape)
print(X.shape)
X['const'] = 1
X_train = X[:15000]
Y_train = Y[:15000]
X_test = X[15000:]
Y_test = Y[15000:]
# scaler.fit(X_train[['num_legs_from_borderalgo','angle','oneway']])
# X_train[['num_legs_from_borderalgo','angle','oneway']] = scaler.transform(X_train[['num_legs_from_borderalgo','angle','oneway']])
# X_test[['num_legs_from_borderalgo','angle','oneway']] = scaler.transform(X_test[['num_legs_from_borderalgo','angle','oneway']])


# pdb.set_trace()
print(X_test.shape)

# print(len(X))
# clf.fi
result = clf.fit(X_train, Y_train)
print("Train score : " + str(clf.score(X_train, Y_train)))
print("Test score : " + str(clf.score(X_test, Y_test)))

# print(X.columns)
print(result.coef_)
# print(clf.predict(X_test))
# plt.hist(clf.predict(X_test),bins=10)
# plt.show()
# print(clf.predict(X_test).std())
# print(clf.predict(X_test))
plt.hist(Y_test-clf.predict(X_test),bins=25)
resids = Y_test-clf.predict(X_test)

best_fit_line = scipy.stats.norm.pdf(np.linspace(-100,300,1000), resids.mean(), resids.std(ddof=1))*8000

plt.plot(np.linspace(-100,300,1000), best_fit_line)
# plt.yscale('log')
# plt.xlim(-5,30)
plt.title("Residuals of truth-prediction")
plt.show()
print("std: " + str((Y_test-clf.predict(X_test)).std(ddof=1)))
print("mean: " + str((Y_test-clf.predict(X_test)).mean() ))
plt.hist(Y_test - Y_test.mean(),bins=35)
plt.title("Residuals using mean as prediction.")
plt.show()
# print((Y_test - Y_test.mean()).mean())
print("std using mean as prediction :" +str((Y_test - Y_test.mean()).std(ddof=1)))

plt.hist(Y_train-clf.predict(X_train),bins=35)
plt.title('Residuals of truth-prediction for training data')
# plt.yscale('log')
# plt.xlim(-5,30)
plt.show()


In [ ]:
plt.hist(clf.predict(X))

In [ ]:
plt.hist(Y_test,bins=30)
plt.show()

# I ad-hoc create the AI prediction columns in the df_int dataframe for export


In [ ]:
# df_int['AI_fatal_crash_rate'] = clf.predict(X)/12.8

In [ ]:
# df_int['AI_fatal_rate_err'] = 0.0125

In [ ]:
# df_int.to_csv("Intersections_withCrashRates_WithAI_Preds.csv")

In [ ]:
# [x for x in clf.predict(X_test)]
# X_test['']

# Only scratchpad work below this line.

In [ ]:
plt.figure(figsize=(13,10))
bins = plt.hist( [ x*(10/12.8) for x in Y_test ] , rwidth=0.95)
# plt.xlim(1,6)
plt.yscale('log')
plt.xlabel("# of major injury crashes in 10 years",fontsize=22)

In [ ]:
X_test['prediction'] = clf.predict(X_test)

In [ ]:
plt.figure(figsize=(13,10))
(X_test['prediction']*(10/12.8)).hist(bins=8,rwidth=0.95)
plt.yscale('log')
plt.xlabel("Predicted # major injury crashes in 10 years",fontsize=22)

In [ ]:
X_safe = X_test[ X_test['prediction'] < 20 ]
X_danger = X_test[ X_test['prediction'] > 20]

In [ ]:
X_safe.describe()

In [ ]:
X_danger.describe()

In [ ]:
# Is there a significant diference between angles?
# Z = (mu_1 - mu_2 / sqrt(sigma_1^2 + sigma_2^2))
Z = (1.525520 - 1.477102) / np.sqrt(0.460798**2 + 0.177618**2)
print(Z)
# No

In [ ]:
df_int['num_legs_from_borderalgo'].apply(cast_to_float)

In [ ]:
np.corrcoef(df_int['oneway'].apply(cast_to_float),df_int['crash_count'].astype(float))

In [ ]:
df_int.to_csv('crash_model_dataframe.csv')

In [ ]:
!pwd

In [ ]:
# # pd.DataFrame()
# severe_columns = [x for x in df_crashes.columns if "FATAL" in x.upper() or "MAJOR" in x.upper()]
# df_crashes_fatal = df_crashes[ pd.DataFrame.any(df_crashes[severe_columns].astype(int) > 0,axis=1) ]

#     df_int_dict = df_int.to_dict('records')
# df_crashes_dict = df_crashes_fatal.to_dict('records')

# crash_mapping = []

# print("Beginning loop...")
# for i, intersection in enumerate(df_int_dict):
#     crash_count = 0
#     crash_ids = []
#     for j, crash in enumerate(df_crashes_dict):
#         distance = geo_distance((intersection['latitude'],intersection['longitude']),
#                                 (crash['latitude'],crash['longitude']))
        
#         radius = distance.m
        
#         if radius < I 50:
#             crash_count += 1
#             crash_ids.append(crash['objectid'])
#     print("Intersection #: " + str(i))
#     print("crash_count:" + str(crash_count))
#     crash_mapping.append((intersection['nodeid'],crash_ids)) 
    
#     if i > 1000:
#         break
    
# crashes

# Below is from an earlier exploration on poisson fits
The data is two skewed to fit to a poisson. Once the data is normalized by traffic volume this may be worth revisiting.

In [ ]:
# def is_severe(row):
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import poisson

In [ ]:
def _ll_poisson(y, X, beta, alph):
    """
    Poisson = (lambda^N*exp(-lambda))/N!
    """
    mu = np.exp(np.dot(X, beta))
    size = 1/alph
    prob = size/(size+mu)
    ll = nbinom.logpmf(y, size, prob)
    ll = poisson.logpmf(y,)
    return ll

class Poisson(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(NBin, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self, params):
        alph = params[-1]
        beta = params[:-1]
        ll = _ll_nb2(self.endog, self.exog, beta, alph)
        return -ll

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('alpha')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), .5)
            # intercept
            start_params[-2] = np.log(self.endog.mean())
        return super(NBin, self).fit(start_params=start_params,
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.special import factorial
from scipy import stats
import copy

def poisson(k, lamb):
    """poisson pdf, parameter lamb is the fit parameter"""
    return (lamb**k/factorial(k)) * np.exp(-lamb)


def negative_log_likelihood(params, data):
    """
    The negative log-Likelihood-Function
    """

    lnl = - np.sum(np.log(poisson(data, params[0])))
    return lnl

def negative_log_likelihood(params, data):
    ''' better alternative using scipy '''
    return -stats.poisson.logpmf(data, params[0]).sum()


# get poisson deviated random numbers
# data = np.random.poisson(1.2, 1000)
data = df_int[df_int['num_legs'].apply(cast_to_float)==4]['crash_count'].apply(cast_to_float).dropna()
print(data)

# minimize the negative log-Likelihood

result = minimize(negative_log_likelihood,  # function to minimize
                  x0=np.ones(1),            # start value
                  args=(data,),             # additional arguments for function
                  method='Powell',          # minimization method, see docs
                  )
# result is a scipy optimize result object, the fit parameters 
# are stored in result.x
print(result)
# print(dir(result))
func_min = result.fun

scan_value = func_min
scan_parameter = copy.deepcopy(result.x)
while scan_value < 2*func_min:
    scan_value = negative_log_likelihood(scan_parameter,data)
    scan_parameter[0] += 0.2
print("1Sigma value is :" )
print(scan_parameter)
print(scan_value)
    
# plot poisson-distribution with fitted parameter
x_plot = np.arange(0, 35)

plt.plot(
    x_plot,
    stats.poisson.pmf(x_plot, result.x[0]),
    marker='o', linestyle='',
    label='Fit result',
)
plt.plot(
    x_plot,
    stats.poisson.pmf(x_plot, scan_parameter[0]),
    marker='x', linestyle='',
    label='Uncertainty result'
)
plt.hist(df_int['crash_count'].apply(cast_to_float),density=True,bins=35,label='Data')
plt.legend()
plt.show()

In [ ]:
stats.poisson.pmf(x_plot, scan_value)

In [ ]:
index = df_int['crash_count'].apply(cast_to_float).dropna().index
df_int.loc[index,'crash_count'].apply(cast_to_float)
df_int.loc[index][['num_legs_from_borderalgo','angle']].astype(float)

In [ ]:
np.sqrt(clf.family.unit_variance(3))

In [ ]:
import tweedie, seaborn as sns, matplotlib.pyplot as plt

mu = 3.5
phi = np.sqrt(clf.family.unit_variance(mu)/mu**1.5)
phi = np.sqrt(clf.family.unit_variance(mu))


tvs = tweedie.tweedie(mu=mu, p=1.5, phi=phi).rvs(10000)
plt.hist(tvs,bins=50,density=True)
# plt.yscale('log')
# ax = sns.kdeplot(tvs,bw=0.05)
plt.show()

In [ ]:
rvs = tweedie.tweedie.rvs(1.5,5,3,size=40)
results = tweedie.tweedie.fit(rvs)

In [ ]:
results

In [ ]:
plt.plot( tweedie.tweedie.pdf(np.linspace(0,100,num=60),results[-1],results[1],results[2]) )
plt.show()

In [ ]:
plt.plot( tweedie.tweedie.pdf(np.linspace(0,100,num=60),1.5,5,3) )
plt.show()